In [1]:
%matplotlib inline
import h5py
import numpy as np
import os
from UG2.utils import data as data_utils
from UG2.utils import image as image_utils
from UG2.config import Config
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import scipy.ndimage as im
import time
from UG2 import main

In [2]:
data_path            = "/data/UG2_data/h5Files/"
traindedModels_path  = "/data/UG2_data/trainedModels/"

In [3]:
## create blurred-dataset 
kernel_blurring_flag= True
num_files           = 10
factor              = 1
patch_size          = np.array([224, 224])
epochsList          = [50]
learningRateList    = [0.001]
numberTrainingKernel= [1]
flags               = [0,1]
numberOfExperiments = len(epochsList)*len(learningRateList)
dataset_name        = "imagenet"

In [4]:
trainStartIndex = 0
numberCompletedExperiments = 0
fileStartIndex  = 1
for i in range(trainStartIndex, len(epochsList)):
    for nLR in learningRateList:
        for nTK in numberTrainingKernel:
            config = Config() 
            config.epochs = epochsList[i]
            config.cuda = True
            config.data_parallel = True
            config.lr = nLR
            config.data_path = data_path
            config.data_files = []
            for fileIndex in range(fileStartIndex,num_files+1):
                if kernel_blurring_flag:
                    
                    config.data_files.append(os.path.join(data_path, dataset_name+"_nTK_"+str(nTK)+ "_flag_" + "_".join([str(f) for f in flags])+"factor"+str(factor)+"nFile"+str(fileIndex)+".h5"))
                else:        
                    config.data_files.append(os.path.join(data_path, dataset_name+".h5"))
                    
            config.batch_size = 50
            config.print_step = 5
            config.checkpoint = 20
            config.exit_loss_window = 5
            config.loss_epsilon = 0.01
            config.resume_training_flag = False
            config.kernel_blurring_flag = False
            config.discriminator = "feat_ext"
            config.factor      = factor
            config.img_size    = 64
            config.num_workers = 8
            config.data_format = "h5"
            
            if i:
                config.epochs = epochsList[i]-epochsList[i-1]
                config.resume_training_flag = True
                config.resume_model_path = traindedModels_path
                config.resume_model_name = "modellrRate"+str(nLR).replace(".","_")+"nEpochs"+str(epochsList[i-1])+".pth"

            config.model_path = traindedModels_path
            config.model_name = "modellrRate"+str(nLR).replace(".","_")+"nEpochs"+str(epochsList[i])+"nTK"+str(nTK)+"nFile"+str(num_files)+".pth"
            numberCompletedExperiments = numberCompletedExperiments+1
            main.train(config)
        
            print("Completed training on file: "+str(num_files))        

time:  29.747249603271484  Error:  1.04325358629
saved checkpoint at epoch:  0
time:  29.80924153327942  Error:  0.46864510376
time:  29.85612440109253  Error:  0.399832420908
time:  29.897969007492065  Error:  0.360147234164
time:  29.972456693649292  Error:  0.332988003418
saved checkpoint at epoch:  20
time:  29.88928484916687  Error:  0.314876724593
time:  29.84264588356018  Error:  0.301519715879
time:  29.65872573852539  Error:  0.292111550421
time:  29.672359704971313  Error:  0.283822483588
saved checkpoint at epoch:  40
time:  29.656681299209595  Error:  0.277311695758
Model saved as:  modellrRate0_001nEpochs50nTK1nFile10.pth
Completed training on file: 10
